In [2]:
import random
from tensorflow.keras.optimizers import SGD
from keras.layers import Dense, Dropout
from keras.models import load_model
from keras.models import Sequential
import numpy as np
import pickle
import json
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
nltk.download('omw-1.4')
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Zach\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Zach\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Zach\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
# init file
words = []
classes = []
documents = []
ignore_words = ["?", "!"]
data_file = open("intentsmql.json").read()
intents = json.loads(data_file)


In [5]:
# words
for intent in intents["intents"]:
    for pattern in intent["patterns"]:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent["tag"]))

        # adding classes to our class list
        if intent["tag"] not in classes:
            classes.append(intent["tag"])

In [6]:
# lemmatizer
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print(len(documents), "documents")

print(len(classes), "classes", classes)

print(len(words), "unique lemmatized words", words)


pickle.dump(words, open("words_mql.pkl", "wb"))
pickle.dump(classes, open("classes_mql.pkl", "wb"))


59 documents
15 classes ['', 'bot1', 'dev21', 'emplacement', 'events', 'fav', 'goodbye', 'greetings', 'name', 'name1', 'need', 'noanswer', 'programming', 'thanks', 'who']
68 unique lemmatized words ['7taj', 'ach', 'alaikom', 'ana', 'atos', 'bach', 'baslama', 'bdat', 'beslama', 'blasst', 'bot', 'bye', 'cap', 'capgemini', 'cc', 'cdi', 'cgi', 'chekoun', 'chkon', 'chokran', 'concour', 'coordonateur', 'cv', 'debut', 'deloite', 'ee', 'filiere', 'fin', 'good', 'hi', 'imta', 'integer', 'java', 'jee', 'kayn', 'kidayr', 'lah', 'le', 'licence', 'localisation', 'mdevelopink', 'mql', 'multinational', 'ndkhol', 'nta', 'okay', 'parrain', 'pfe', 'position', 'programmation', 'qui', 'salam', 'salamo', 'salut', 'slm', 'smitk', 'sqli', 'stage', "t'es", 'tarikh', 'thala', 'thank', 'thanks', 'thankyou', 'wesh', 'wow', 'yhfedak', 'you']


In [7]:
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:, 0])
train_y = list(training[:, 1])
print("Training data created")


Training data created


C:\Users\Zach\AppData\Local\Temp\ipykernel_9344\115763920.py:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [8]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation="softmax"))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               8832      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 15)                975       
                                                                 
Total params: 18,063
Trainable params: 18,063
Non-trainable params: 0
_________________________________________________________________


In [23]:
from tensorflow.keras.optimizers import Adam

adam = Adam(learning_rate=0.001)
model.compile(loss='mean_squared_error', optimizer=adam)

In [24]:
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save("chatbot_model_mql.h5", hist)
print("model created")


Epoch 1/200
12/12 [==============================] - 1s 1ms/step - loss: 0.0605
Epoch 2/200
12/12 [==============================] - 0s 1ms/step - loss: 0.0599
Epoch 3/200
12/12 [==============================] - 0s 2ms/step - loss: 0.0600
Epoch 4/200
12/12 [==============================] - 0s 2ms/step - loss: 0.0598
Epoch 5/200
12/12 [==============================] - 0s 1ms/step - loss: 0.0592
Epoch 6/200
12/12 [==============================] - 0s 2ms/step - loss: 0.0593
Epoch 7/200
12/12 [==============================] - 0s 398us/step - loss: 0.0580
Epoch 8/200
12/12 [==============================] - 0s 791us/step - loss: 0.0583
Epoch 9/200
12/12 [==============================] - 0s 857us/step - loss: 0.0573
Epoch 10/200
12/12 [==============================] - 0s 2ms/step - loss: 0.0560
Epoch 11/200
12/12 [==============================] - 0s 2ms/step - loss: 0.0570
Epoch 12/200
12/12 [==============================] - 0s 2ms/step - loss: 0.0569
Epoch 13/200
12/12 [===========

12/12 [==============================] - 0s 2ms/step - loss: 0.0064
Epoch 102/200
12/12 [==============================] - 0s 2ms/step - loss: 0.0064
Epoch 103/200
12/12 [==============================] - 0s 2ms/step - loss: 0.0064
Epoch 104/200
12/12 [==============================] - 0s 102us/step - loss: 0.0053
Epoch 105/200
12/12 [==============================] - 0s 212us/step - loss: 0.0055
Epoch 106/200
12/12 [==============================] - 0s 757us/step - loss: 0.0068
Epoch 107/200
12/12 [==============================] - 0s 856us/step - loss: 0.0049
Epoch 108/200
12/12 [==============================] - 0s 2ms/step - loss: 0.0058
Epoch 109/200
12/12 [==============================] - 0s 2ms/step - loss: 0.0062
Epoch 110/200
12/12 [==============================] - 0s 2ms/step - loss: 0.0061
Epoch 111/200
12/12 [==============================] - 0s 1ms/step - loss: 0.0071
Epoch 112/200
12/12 [==============================] - 0s 1ms/step - loss: 0.0055
Epoch 113/200
12/12 [=

12/12 [==============================] - 0s 2ms/step - loss: 0.0028
model created


In [21]:
train_x.shape

AttributeError: 'list' object has no attribute 'shape'